Ejercicio 2. Spark SQL (5 puntos)

 Carga el fichero cars.csv proporcionado en Colab o Databricks .
 En el fichero cars.csv tenemos información sobre las características técnicas de varios vehículos:

 • Carga el fichero en un DataFrame de Spark que se llame cars, muestra por pantalla la estructura de cars y las primeras 5 filas (0,5 puntos)

 • Obtener las columnas “Car” y “Cylinders” de todos los que sean de Europa (1 punto) (0,5 puntos)

 • Obtener el número de vehículos por “Origin” (0,5 puntos) • Obtener el número de vehículos por “Origin” y “Model” y ordenar el resultado por el número de vehículos de manera descendente (1 punto)

  • Obtén la media de "Horsepower", "Weight" y "Acceleration" por "Origen" (1 punto)
  
  • Calcula la ratio entre potencia y peso y a continuación sacar la media por cantidad de cilindros (1,5 puntos)

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os # libreria de manejo del sistema operativo
os.system("wget -q https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz")
os.system("tar xf /spark-3.4.3-bin-hadoop3.tgz")
# instalar pyspark
!tar xf /content/spark-3.4.3-bin-hadoop3.tgz
!pip install -q pyspark

# Variables de Entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.4.3-bin-hadoop3"

!pip install -q findspark
import findspark
#We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.
findspark.init()
#Para saber donde está instalado
findspark.find()

'/content/spark-3.4.3-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession
#Creamos el contexto SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("SparkSQL Colab")\
        .getOrCreate()
#A partir del SparkSession puedo obtener el SparkContext
sc = spark.sparkContext

Cargar el fichero en un DataFrame de Spark que se llame cars

In [14]:
# Cargar el archivo CSV con el delimitador correcto
cars = spark.read.option("delimiter", ";").option("header", "true").csv("/content/cars.csv")

# Mostrar la estructura de cars
cars.printSchema()
cars.printSchema()

# Mostrar las primeras 5 filas
cars.show(5)

root
 |-- Car: string (nullable = true)
 |-- MPG: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Displacement: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Origin: string (nullable = true)

root
 |-- Car: string (nullable = true)
 |-- MPG: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Displacement: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Origin: string (nullable = true)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+---

Obtener las columnas "Car" y "Cylinders" de todos los que sean de Europa

In [15]:
europe_cars = cars.filter(cars.Origin == "Europe").select("Car", "Cylinders")
europe_cars.show()


+--------------------+---------+
|                 Car|Cylinders|
+--------------------+---------+
|Citroen DS-21 Pallas|        4|
|Volkswagen 1131 D...|        4|
|         Peugeot 504|        4|
|         Audi 100 LS|        4|
|            Saab 99e|        4|
|            BMW 2002|        4|
|Volkswagen Super ...|        4|
|           Opel 1900|        4|
|         Peugeot 304|        4|
|           Fiat 124B|        4|
|Volkswagen Model 111|        4|
|   Volkswagen Type 3|        4|
|     Volvo 145e (sw)|        4|
| Volkswagen 411 (sw)|        4|
|    Peugeot 504 (sw)|        4|
|     Renault 12 (sw)|        4|
|Volkswagen Super ...|        4|
|Fiat 124 Sport Coupe|        4|
|            Fiat 128|        4|
|          Opel Manta|        4|
+--------------------+---------+
only showing top 20 rows



Obtener el número de vehículos por "Origin"




In [16]:
cars.groupBy("Origin").count().show()


+------+-----+
|Origin|count|
+------+-----+
|Europe|   73|
|    US|  254|
| Japan|   79|
+------+-----+



Obtener el número de vehículos por "Origin" y "Model" y ordenar el resultado por el número de vehículos de manera descendente

In [17]:
from pyspark.sql.functions import col

cars.groupBy("Origin", "Model").count().orderBy(col("count").desc()).show()


+------+-----+-----+
|Origin|Model|count|
+------+-----+-----+
|    US|   73|   29|
|    US|   70|   27|
|    US|   79|   23|
|    US|   76|   22|
|    US|   78|   22|
|    US|   75|   20|
|    US|   82|   20|
|    US|   71|   20|
|    US|   72|   18|
|    US|   77|   18|
|    US|   74|   15|
|    US|   81|   13|
| Japan|   80|   13|
| Japan|   81|   12|
|Europe|   80|    9|
| Japan|   82|    9|
|Europe|   76|    8|
| Japan|   78|    8|
|    US|   80|    7|
|Europe|   73|    7|
+------+-----+-----+
only showing top 20 rows



Obtener la media de "Horsepower", "Weight" y "Acceleration" por "Origen"

In [18]:
from pyspark.sql.functions import avg

cars.groupBy("Origin").agg(
    avg("Horsepower").alias("Avg_Horsepower"),
    avg("Weight").alias("Avg_Weight"),
    avg("Acceleration").alias("Avg_Acceleration")
).show()


+------+------------------+------------------+------------------+
|Origin|    Avg_Horsepower|        Avg_Weight|  Avg_Acceleration|
+------+------------------+------------------+------------------+
|Europe| 78.78082191780823|2431.4931506849316| 16.82191780821918|
|    US|118.01181102362204| 3372.700787401575|14.942519685039361|
| Japan| 79.83544303797468| 2221.227848101266|16.172151898734175|
+------+------------------+------------------+------------------+



Calcular la ratio entre potencia y peso y a continuación sacar la media por cantidad de cilindros

In [19]:
from pyspark.sql.functions import expr

# Crear la columna de ratio entre potencia y peso
cars_with_ratio = cars.withColumn("Power_Weight_Ratio", expr("Horsepower / Weight"))

# Sacar la media por cantidad de cilindros
cars_with_ratio.groupBy("Cylinders").agg(avg("Power_Weight_Ratio").alias("Avg_Power_Weight_Ratio")).show()


+---------+----------------------+
|Cylinders|Avg_Power_Weight_Ratio|
+---------+----------------------+
|        3|  0.041441818290915214|
|        8|   0.03872856292574305|
|        5|  0.026973551761855744|
|        6|   0.03163238614784404|
|        4|  0.033217183371144564|
+---------+----------------------+

